In [2]:
from sentence_transformers import SentenceTransformer
import requests
import json
import pandas as pd

In [3]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'

In [4]:
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [5]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [6]:
df = df.iloc[:300]

In [7]:
model_name = 'multi-qa-mpnet-base-dot-v1'

In [8]:
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

d:\anaconda3\envs\llm-zoomcamp\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ganes\.cache\huggingface\hub\models--sentence-transformers--multi-qa-mpnet-base-dot-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
answer_llm = df.iloc[0].answer_llm

In [14]:
embedding_model.encode(answer_llm)[0]

-0.42244655

In [22]:
evaluations = []

for _, row in df.iterrows():
    answer_llm_embedding = embedding_model.encode(row.answer_llm)
    answer_orig_embedding = embedding_model.encode(row.answer_orig)
    
    score = np.dot(answer_llm_embedding, answer_orig_embedding)
    evaluations.append(score)

In [35]:
import numpy as np
percentile_75 = np.percentile(evaluations, 75)
percentile_75

31.674309253692627

In [24]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [33]:
evaluations_norm = []

for _, row in df.iterrows():
    answer_llm_embedding = embedding_model.encode(row.answer_llm)
    answer_orig_embedding = embedding_model.encode(row.answer_orig)
    
    answer_llm_embedding_norm = normalize_vector(answer_llm_embedding)
    answer_orig_embedding_norm = normalize_vector(answer_orig_embedding)
    
    score = np.dot(answer_llm_embedding_norm, answer_orig_embedding_norm)
    evaluations_norm.append(score)

In [34]:
percentile_norm_75 = np.percentile(evaluations_norm, 75)

In [36]:
percentile_norm_75

0.8362348079681396

In [44]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df.iloc[10]

In [45]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

In [46]:
rouge_1_f1 = scores['rouge-1']['f']

In [47]:
rouge_1_f1

0.45454544954545456

In [49]:
average_rouge = np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])

In [50]:
average_rouge

0.35490034990035496

In [51]:
rouge_scores = []

for _, row in df.iterrows():
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    
    rouge_1 = scores['rouge-1']['f']
    rouge_2 = scores['rouge-2']['f']
    rouge_l = scores['rouge-l']['f']
    rouge_avg = np.mean([rouge_1, rouge_2, rouge_l])
    
    rouge_scores.append({
        'rouge_1': rouge_1,
        'rouge_2': rouge_2,
        'rouge_l': rouge_l,
        'rouge_avg': rouge_avg
    })


In [52]:
rouge_df = pd.DataFrame(rouge_scores)

In [53]:
average_rouge_2 = rouge_df['rouge_2'].mean()

In [54]:
average_rouge_2

0.20696501983423318